In [1]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from torchvision import transforms

from trainer import fit
import numpy as np

cuda = torch.cuda.is_available()

# TRIPLET

In [2]:
# Set up data loaders
from datasets import TripletDataset

root_dir = '../AIC20_ReID/image_train'
train_csv = 'metadata/cls_train.csv'
val_csv = 'metadata/cls_val.csv'
label_json = 'metadata/train_image_metadata.json'

size = (224, 224)

triplet_train_dataset = TripletDataset(root_dir, train_csv, label_json,
                                       transform = transforms.Compose([
                                        transforms.Resize(size),  
                                        transforms.ToTensor()
                                      ]))
triplet_val_dataset = TripletDataset(root_dir, val_csv, label_json,
                                     transform = transforms.Compose([
                                        transforms.Resize(size),
                                        transforms.ToTensor()
                                      ]))

batch_size = 8
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_val_loader = torch.utils.data.DataLoader(triplet_val_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:
# Set up the network and training parameters
from networks import EfficientNetExtractor, TripletNet
from losses import TripletLoss

margin = 1.
embedding_net = EfficientNetExtractor('b4')
model = TripletNet(embedding_net)

if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

In [ ]:
fit(triplet_train_loader, triplet_val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

/home/cuong/AIC20-Track2/venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/25585 (0%)]	Loss: 24.244934


In [ ]:
torch.save(model, 'weights/triplet-b4-200404.pth')

# ONLINE TRIPLET LOSS

In [2]:
# Set up data loaders
from datasets import ImageFolderDataset
from datasets import BalancedBatchSampler

import csv
import json

root_dir = '../AIC20_ReID/image_train'
train_csv = 'metadata/cls_train.csv'
val_csv = 'metadata/cls_val.csv'
label_json = 'metadata/train_image_metadata.json'

size = (224, 224)

def get_images_labels(vehicle_csv, label_json):
    with open(label_json, 'r') as json_file:
        data_dict = json.load(json_file)
    image_names = []
    labels = []
    with open(vehicle_csv, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)
        for row in csv_reader:
            vehicle_id = row[0]
            for cam_id in data_dict[vehicle_id]:
                image_names += [image_name for image_name in data_dict[vehicle_id][cam_id]]
                labels += [int(vehicle_id) for image_name in data_dict[vehicle_id][cam_id]]
    return image_names, labels

train_image_names, train_labels = get_images_labels(train_csv, label_json) 
val_image_names, val_labels = get_images_labels(val_csv, label_json) 

train_dataset = ImageFolderDataset(root_dir, train_image_names, train_labels,
                                       transform = transforms.Compose([
                                        transforms.Resize(size),  
                                        transforms.ToTensor()
                                      ]))
val_dataset = ImageFolderDataset(root_dir, val_image_names, val_labels,
                                     transform = transforms.Compose([
                                        transforms.Resize(size),
                                        transforms.ToTensor()
                                      ]))



train_batch_sampler = BalancedBatchSampler(train_dataset.labels, n_classes=5, n_samples=5)
val_batch_sampler = BalancedBatchSampler(val_dataset.labels, n_classes=5, n_samples=5)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_val_loader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_batch_sampler, **kwargs)

In [ ]:
# Set up the network and training parameters
from networks import EfficientNetExtractor
from losses import OnlineTripletLoss
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric

margin = 1.
embedding_net = EfficientNetExtractor('b4')
model = embedding_net

if cuda:
    model.cuda()
loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

Loaded pretrained weights for efficientnet-b4


In [ ]:
fit(online_train_loader, online_val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

/home/cuong/AIC20-Track2/venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/25585 (0%)]	Loss: 53.939754	Average nonzero triplets: 47.0
Train: [2500/25585 (10%)]	Loss: 11.390418	Average nonzero triplets: 41.24752475247525
Train: [5000/25585 (20%)]	Loss: 4.701021	Average nonzero triplets: 35.90049751243781
Train: [7500/25585 (29%)]	Loss: 4.039120	Average nonzero triplets: 32.08305647840532
Train: [10000/25585 (39%)]	Loss: 3.847635	Average nonzero triplets: 29.576059850374065
Train: [12500/25585 (49%)]	Loss: 2.920765	Average nonzero triplets: 27.41117764471058
Train: [15000/25585 (59%)]	Loss: 2.678268	Average nonzero triplets: 25.798668885191347
Train: [17500/25585 (68%)]	Loss: 2.041569	Average nonzero triplets: 24.546362339514978
Train: [20000/25585 (78%)]	Loss: 1.928217	Average nonzero triplets: 23.621722846441948
Train: [22500/25585 (88%)]	Loss: 1.621420	Average nonzero triplets: 22.67258601553829
Train: [25000/25585 (98%)]	Loss: 1.750617	Average nonzero triplets: 22.072927072927072
Epoch: 1/20. Train set: Average loss: 3.6900	Average nonzero triplet

In [ ]:
torch.save(model, 'weights/onlinetriplet-b4-200406-semihardest.pth')